In [1]:
import os
for dirname, _, filenames in os.walk("../../weights"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../../weights/deberta_v3_base_512_reinit_pseudo3/last_epoch_fold4.ckpt
../../weights/deberta_v3_base_512_reinit_pseudo3/last_epoch_fold2.ckpt
../../weights/deberta_v3_base_512_reinit_pseudo3/last_epoch_fold1.ckpt
../../weights/deberta_v3_base_512_reinit_pseudo3/last_epoch_fold3.ckpt
../../weights/deberta_v3_base_512_reinit_pseudo3/last_epoch_fold0.ckpt
../../weights/deberta_v3_base_4096_reinit_pseudo/last_epoch_fold4.ckpt
../../weights/deberta_v3_base_4096_reinit_pseudo/last_epoch_fold2.ckpt
../../weights/deberta_v3_base_4096_reinit_pseudo/last_epoch_fold1.ckpt
../../weights/deberta_v3_base_4096_reinit_pseudo/last_epoch_fold3.ckpt
../../weights/deberta_v3_base_4096_reinit_pseudo/last_epoch_fold0.ckpt


Imports

In [2]:
import gc

import sys
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import sklearn.metrics as metrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml 
import joblib
from sklearn import linear_model
from pytorch_lightning import Trainer, seed_everything
from scipy.optimize import minimize
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
)

config

In [3]:
with open("./configs/deberta_v3_base_512_reinit.yaml", encoding="utf-8") as f:
    cfg1 = yaml.safe_load(f)
cfg1["general"]["wandb_desabled"] = True
cfg1["model_save"] = False
cfg1["enable_checkpointing"] = False
cfg1["pl_params"]["precision"] = 32
#cfg1["model_name"] = "../input/deberta-v3-base"
cfg1['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit"

with open("./configs/deberta_v3_base_4096_reinit.yaml", encoding="utf-8") as f:
    cfg2 = yaml.safe_load(f)
cfg2["general"]["wandb_desabled"] = True
cfg2["model_save"] = False
cfg2["enable_checkpointing"] = False
cfg2["pl_params"]["precision"] = 32
#cfg2["model_name"] = "../input/deberta-v3-base"
cfg2['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit"

with open("./configs/deberta_v3_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg3 = yaml.safe_load(f)
cfg3["general"]["wandb_desabled"] = True
cfg3["model_save"] = False
cfg3["enable_checkpointing"] = False
cfg3["pl_params"]["precision"] = 32
#cfg3["model_name"] = "../input/deberta-v3-large"
cfg3['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit"

with open("./configs/deberta_v3_large_4096_reinit.yaml", encoding="utf-8") as f:
    cfg4 = yaml.safe_load(f)
cfg4["general"]["wandb_desabled"] = True
cfg4["model_save"] = False
cfg4["enable_checkpointing"] = False
cfg4["pl_params"]["precision"] = 32
#cfg4["model_name"] = "../input/deberta-v3-large"
cfg4['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit"

with open("./configs/deberta_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg5 = yaml.safe_load(f)
cfg5["general"]["wandb_desabled"] = True
cfg5["model_save"] = False
cfg5["enable_checkpointing"] = False
cfg5["pl_params"]["precision"] = 32
#cfg5["model_name"] = "../input/deberta-large"
cfg5['save_weight_folder'] = "../../weights/deberta_large_512_reinit"

with open("./configs/muppet_large_512_reinit.yaml", encoding="utf-8") as f:
    cfg6 = yaml.safe_load(f)
cfg6["general"]["wandb_desabled"] = True
cfg6["model_save"] = False
cfg6["enable_checkpointing"] = False
cfg6["pl_params"]["precision"] = 32
#cfg6["model_name"] = "../input/muppet-roberta-large"
cfg6['save_weight_folder'] = "../../weights/muppet_large_512_reinit"

with open("./configs/deberta_xlarge_512_reinit.yaml", encoding="utf-8") as f:
    cfg7 = yaml.safe_load(f)
cfg7["general"]["wandb_desabled"] = True
cfg7["model_save"] = False
cfg7["enable_checkpointing"] = False
cfg7["pl_params"]["precision"] = 32
#cfg7["model_name"] = "../input/debertaxlarge"
cfg7['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit"

with open("./configs/mpnet_base_512_reinit.yaml", encoding="utf-8") as f:
    cfg8 = yaml.safe_load(f)
cfg8["general"]["wandb_desabled"] = True
cfg8["model_save"] = False
cfg8["enable_checkpointing"] = False
cfg8["pl_params"]["precision"] = 32
cfg8['save_weight_folder'] = "../../weights/mpnet_base_512_reinit"

with open("./configs/deberta_v3_small_512_reinit.yaml", encoding="utf-8") as f:
    cfg9 = yaml.safe_load(f)
cfg9["general"]["wandb_desabled"] = True
cfg9["model_save"] = False
cfg9["enable_checkpointing"] = False
cfg9["pl_params"]["precision"] = 32
cfg9['save_weight_folder'] = "../../weights/deberta_v3_small_512_reinit"

with open("./configs/deberta_v3_xsmall_512_reinit.yaml", encoding="utf-8") as f:
    cfg10 = yaml.safe_load(f)
cfg10["general"]["wandb_desabled"] = True
cfg10["model_save"] = False
cfg10["enable_checkpointing"] = False
cfg10["pl_params"]["precision"] = 32
cfg10['save_weight_folder'] = "../../weights/deberta_v3_xsmall_512_reinit"

In [4]:
with open("./configs/deberta_v3_base_512_reinit_pseudo3.yaml", encoding="utf-8") as f:
    cfg1p = yaml.safe_load(f)
cfg1p["general"]["wandb_desabled"] = True
cfg1p["model_save"] = False
cfg1p["enable_checkpointing"] = False
cfg1p["pl_params"]["precision"] = 32
#cfg1p["model_name"] = "../input/deberta-v3-base"
cfg1p['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_pseudo"

with open("./configs/deberta_v3_base_4096_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg2p = yaml.safe_load(f)
cfg2p["general"]["wandb_desabled"] = True
cfg2p["model_save"] = False
cfg2p["enable_checkpointing"] = False
cfg2p["pl_params"]["precision"] = 32
#cfg2p["model_name"] = "../input/deberta-v3-base"
cfg2p['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit_pseudo"

with open("./configs/deberta_v3_large_512_reinit_pseudo3.yaml", encoding="utf-8") as f:
    cfg3p = yaml.safe_load(f)
cfg3p["general"]["wandb_desabled"] = True
cfg3p["model_save"] = False
cfg3p["enable_checkpointing"] = False
cfg3p["pl_params"]["precision"] = 32
#cfg3p["model_name"] = "../input/deberta-v3-large"
cfg3p['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit_pseudo"

with open("./configs/deberta_v3_large_4096_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg4p = yaml.safe_load(f)
cfg4p["general"]["wandb_desabled"] = True
cfg4p["model_save"] = False
cfg4p["enable_checkpointing"] = False
cfg4p["pl_params"]["precision"] = 32
#cfg4p["model_name"] = "../input/deberta-v3-large"
cfg4p['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit_pseudo"

with open("./configs/deberta_large_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg5p = yaml.safe_load(f)
cfg5p["general"]["wandb_desabled"] = True
cfg5p["model_save"] = False
cfg5p["enable_checkpointing"] = False
cfg5p["pl_params"]["precision"] = 32
#cfg5p["model_name"] = "../input/deberta-large"
cfg5p['save_weight_folder'] = "../../weights/deberta_large_512_reinit_pseudo"

with open("./configs/muppet_large_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg6p = yaml.safe_load(f)
cfg6p["general"]["wandb_desabled"] = True
cfg6p["model_save"] = False
cfg6p["enable_checkpointing"] = False
cfg6p["pl_params"]["precision"] = 32
#cfg6p["model_name"] = "../input/muppet-roberta-large"
cfg6p['save_weight_folder'] = "../../weights/muppet_large_512_reinit_pseudo"

with open("./configs/deberta_xlarge_512_reinit_pseudo.yaml", encoding="utf-8") as f:
    cfg7p = yaml.safe_load(f)
cfg7p["general"]["wandb_desabled"] = True
cfg7p["model_save"] = False
cfg7p["enable_checkpointing"] = False
cfg7p["pl_params"]["precision"] = 32
#cfg7p["model_name"] = "../input/debertaxlarge"
cfg7p['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit_pseudo"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_cls.yaml", encoding="utf-8") as f:
    cfg8p = yaml.safe_load(f)
cfg8p["general"]["wandb_desabled"] = True
cfg8p["model_save"] = False
cfg8p["enable_checkpointing"] = False
cfg8p["pl_params"]["precision"] = 32
#cfg8p["model_name"] = "../input/deberta-v3-base"
cfg8p['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_pseudo3_cls"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_wap_attention.yaml", encoding="utf-8") as f:
    cfg9p = yaml.safe_load(f)
cfg9p["general"]["wandb_desabled"] = True
cfg9p["model_save"] = False
cfg9p["enable_checkpointing"] = False
cfg9p["pl_params"]["precision"] = 32
#cfg9p["model_name"] = "../input/deberta-v3-base"
cfg9p['save_weight_folder'] = "../weights/deberta_v3_base_512_reinit_pseudo3_wap_attention"

with open("./configs/deberta_v3_base_512_reinit_pseudo3_wap_attention_3epoch.yaml", encoding="utf-8") as f:
    cfg10p = yaml.safe_load(f)
cfg10p["general"]["wandb_desabled"] = True
cfg10p["model_save"] = False
cfg10p["enable_checkpointing"] = False
cfg10p["pl_params"]["precision"] = 32
#cfg10p["model_name"] = "../input/deberta-v3-base"
cfg10p['save_weight_folder'] = "../weights/deberta_v3_base_512_reinit_pseudo3_wap_attention_3epoch"

In [5]:
with open("./configs/svr/albert_xxlarge_v2_512_svr.yaml", encoding="utf-8") as f:
    cfg1s = yaml.safe_load(f)
cfg1s["general"]["wandb_desabled"] = True
cfg1s["model_save"] = False
cfg1s["enable_checkpointing"] = False
cfg1s["pl_params"]["precision"] = 16
#cfg1s["model_name"] = "../input/deberta-v3-base"
cfg1s['save_weight_folder'] = "../../weights/albert_xxlarge_v2_512_svr"

with open("./configs/svr/bigbird_roberta_large_4096_svr.yaml", encoding="utf-8") as f:
    cfg2s = yaml.safe_load(f)
cfg2s["general"]["wandb_desabled"] = True
cfg2s["model_save"] = False
cfg2s["enable_checkpointing"] = False
cfg2s["pl_params"]["precision"] = 16
#cfg2s["model_name"] = "../input/deberta-v3-base"
cfg2s['save_weight_folder'] = "../../weights/bigbird_roberta_large_4096_svr"

with open("./configs/svr/deberta_large_512_svr.yaml", encoding="utf-8") as f:
    cfg3s = yaml.safe_load(f)
cfg3s["general"]["wandb_desabled"] = True
cfg3s["model_save"] = False
cfg3s["enable_checkpointing"] = False
cfg3s["pl_params"]["precision"] = 16
#cfg3s["model_name"] = "../input/deberta-v3-base"
cfg3s['save_weight_folder'] = "../../weights/deberta_large_512_svr"

with open("./configs/svr/deberta_v2_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg4s = yaml.safe_load(f)
cfg4s["general"]["wandb_desabled"] = True
cfg4s["model_save"] = False
cfg4s["enable_checkpointing"] = False
cfg4s["pl_params"]["precision"] = 16
#cfg4s["model_name"] = "../input/deberta-v3-base"
cfg4s['save_weight_folder'] = "../../weights/deberta_v2_xlarge_512_svr"

with open("./configs/svr/deberta_v2_xxlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg5s = yaml.safe_load(f)
cfg5s["general"]["wandb_desabled"] = True
cfg5s["model_save"] = False
cfg5s["enable_checkpointing"] = False
cfg5s["pl_params"]["precision"] = 16
#cfg5s["model_name"] = "../input/deberta-v3-base"
cfg5s['save_weight_folder'] = "../../weights/deberta_v2_xxlarge_512_svr"

with open("./configs/svr/deberta_v3_base_4096_svr.yaml", encoding="utf-8") as f:
    cfg6s = yaml.safe_load(f)
cfg6s["general"]["wandb_desabled"] = True
cfg6s["model_save"] = False
cfg6s["enable_checkpointing"] = False
cfg6s["pl_params"]["precision"] = 16
#cfg6s["model_name"] = "../input/deberta-v3-base"
cfg6s['save_weight_folder'] = "../../weights/deberta_v3_base_4096_svr"

with open("./configs/svr/deberta_v3_base_512_svr.yaml", encoding="utf-8") as f:
    cfg7s = yaml.safe_load(f)
cfg7s["general"]["wandb_desabled"] = True
cfg7s["model_save"] = False
cfg7s["enable_checkpointing"] = False
cfg7s["pl_params"]["precision"] = 16
#cfg7s["model_name"] = "../input/deberta-v3-base"
cfg7s['save_weight_folder'] = "../../weights/deberta_v3_base_512_svr"

with open("./configs/svr/deberta_v3_large_4096_svr.yaml", encoding="utf-8") as f:
    cfg8s = yaml.safe_load(f)
cfg8s["general"]["wandb_desabled"] = True
cfg8s["model_save"] = False
cfg8s["enable_checkpointing"] = False
cfg8s["pl_params"]["precision"] = 16
#cfg8s["model_name"] = "../input/deberta-v3-base"
cfg8s['save_weight_folder'] = "../../weights/deberta_v3_large_4096_svr"

with open("./configs/svr/deberta_v3_large_512_svr.yaml", encoding="utf-8") as f:
    cfg9s = yaml.safe_load(f)
cfg9s["general"]["wandb_desabled"] = True
cfg9s["model_save"] = False
cfg9s["enable_checkpointing"] = False
cfg9s["pl_params"]["precision"] = 16
#cfg9s["model_name"] = "../input/deberta-v3-base"
cfg9s['save_weight_folder'] = "../../weights/deberta_v3_large_512_svr"

with open("./configs/svr/deberta_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg10s = yaml.safe_load(f)
cfg10s["general"]["wandb_desabled"] = True
cfg10s["model_save"] = False
cfg10s["enable_checkpointing"] = False
cfg10s["pl_params"]["precision"] = 16
#cfg10s["model_name"] = "../input/deberta-v3-base"
cfg10s['save_weight_folder'] = "../../weights/deberta_xlarge_512_svr"

with open("./configs/svr/electra_large_512_svr.yaml", encoding="utf-8") as f:
    cfg11s = yaml.safe_load(f)
cfg11s["general"]["wandb_desabled"] = True
cfg11s["model_save"] = False
cfg11s["enable_checkpointing"] = False
cfg11s["pl_params"]["precision"] = 16
#cfg11s["model_name"] = "../input/deberta-v3-base"
cfg11s['save_weight_folder'] = "../../weights/electra_large_512_svr"

with open("./configs/svr/funnel_xlarge_512_svr.yaml", encoding="utf-8") as f:
    cfg12s = yaml.safe_load(f)
cfg12s["general"]["wandb_desabled"] = True
cfg12s["model_save"] = False
cfg12s["enable_checkpointing"] = False
cfg12s["pl_params"]["precision"] = 16
#cfg12s["model_name"] = "../input/deberta-v3-base"
cfg12s['save_weight_folder'] = "../../weights/funnel_xlarge_512_svr"

with open("./configs/svr/longformer_large_4096.yaml", encoding="utf-8") as f:
    cfg13s = yaml.safe_load(f)
cfg13s["general"]["wandb_desabled"] = True
cfg13s["model_save"] = False
cfg13s["enable_checkpointing"] = False
cfg13s["pl_params"]["precision"] = 16
#cfg13s["model_name"] = "../input/deberta-v3-base"
cfg13s['save_weight_folder'] = "../../weights/longformer_large_4096"

with open("./configs/svr/muppet_large_512_svr.yaml", encoding="utf-8") as f:
    cfg14s = yaml.safe_load(f)
cfg14s["general"]["wandb_desabled"] = True
cfg14s["model_save"] = False
cfg14s["enable_checkpointing"] = False
cfg14s["pl_params"]["precision"] = 16
#cfg14s["model_name"] = "../input/deberta-v3-base"
cfg14s['save_weight_folder'] = "../../weights/muppet_large_512_svr"

with open("./configs/svr/roberta_large_512_svr.yaml", encoding="utf-8") as f:
    cfg15s = yaml.safe_load(f)
cfg15s["general"]["wandb_desabled"] = True
cfg15s["model_save"] = False
cfg15s["enable_checkpointing"] = False
cfg15s["pl_params"]["precision"] = 16
#cfg15s["model_name"] = "../input/deberta-v3-base"
cfg15s['save_weight_folder'] = "../../weights/roberta_large_512_svr"

In [19]:
with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg1sp = yaml.safe_load(f)
cfg1sp["general"]["wandb_desabled"] = True
cfg1sp["model_save"] = False
cfg1sp["enable_checkpointing"] = False
cfg1sp["pl_params"]["precision"] = 16
#cfg1sp["model_name"] = "../input/deberta-v3-base"
cfg1sp['save_weight_folder'] = "../../weights/deberta_v3_base_4096_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg2sp = yaml.safe_load(f)
cfg2sp["general"]["wandb_desabled"] = True
cfg2sp["model_save"] = False
cfg2sp["enable_checkpointing"] = False
cfg2sp["pl_params"]["precision"] = 16
#cfg2sp["model_name"] = "../input/deberta-v3-base"
cfg2sp['save_weight_folder'] = "../../weights/deberta_v3_base_512_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_large_4096_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg3sp = yaml.safe_load(f)
cfg3sp["general"]["wandb_desabled"] = True
cfg3sp["model_save"] = False
cfg3sp["enable_checkpointing"] = False
cfg3sp["pl_params"]["precision"] = 16
#cfg3sp["model_name"] = "../input/deberta-v3-base"
cfg3sp['save_weight_folder'] = "../../weights/deberta_v3_large_4096_reinit_svr_pseudo"

with open("./configs/svr/deberta_v3_large_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg4sp = yaml.safe_load(f)
cfg4sp["general"]["wandb_desabled"] = True
cfg4sp["model_save"] = False
cfg4sp["enable_checkpointing"] = False
cfg4sp["pl_params"]["precision"] = 16
#cfg4sp["model_name"] = "../input/deberta-v3-base"
cfg4sp['save_weight_folder'] = "../../weights/deberta_v3_large_512_reinit_svr_pseudo"

with open("./configs/svr/deberta_xlarge_512_reinit_svr_pseudo.yaml", encoding="utf-8") as f:
    cfg5sp = yaml.safe_load(f)
cfg5sp["general"]["wandb_desabled"] = True
cfg5sp["model_save"] = False
cfg5sp["enable_checkpointing"] = False
cfg5sp["pl_params"]["precision"] = 16
#cfg5sp["model_name"] = "../input/deberta-v3-base"
cfg5sp['save_weight_folder'] = "../../weights/deberta_xlarge_512_reinit_svr_pseudo"

# 1128追加
with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_wap.yaml", encoding="utf-8") as f:
    cfg6sp = yaml.safe_load(f)
cfg6sp["general"]["wandb_desabled"] = True
cfg6sp["model_save"] = False
cfg6sp["enable_checkpointing"] = False
cfg6sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_wap_all.yaml", encoding="utf-8") as f:
    cfg7sp = yaml.safe_load(f)
cfg7sp["general"]["wandb_desabled"] = True
cfg7sp["model_save"] = False
cfg7sp["enable_checkpointing"] = False
cfg7sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_512_reinit_svr_pseudo_mean_pooling_concatenate.yaml", encoding="utf-8") as f:
    cfg8sp = yaml.safe_load(f)
cfg8sp["general"]["wandb_desabled"] = True
cfg8sp["model_save"] = False
cfg8sp["enable_checkpointing"] = False
cfg8sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_wap.yaml", encoding="utf-8") as f:
    cfg9sp = yaml.safe_load(f)
cfg9sp["general"]["wandb_desabled"] = True
cfg9sp["model_save"] = False
cfg9sp["enable_checkpointing"] = False
cfg9sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_wap_all.yaml", encoding="utf-8") as f:
    cfg10sp = yaml.safe_load(f)
cfg10sp["general"]["wandb_desabled"] = True
cfg10sp["model_save"] = False
cfg10sp["enable_checkpointing"] = False
cfg10sp["pl_params"]["precision"] = 16

with open("./configs/svr/deberta_v3_base_4096_reinit_svr_pseudo_mean_pooling_concatenate.yaml", encoding="utf-8") as f:
    cfg11sp = yaml.safe_load(f)
cfg11sp["general"]["wandb_desabled"] = True
cfg11sp["model_save"] = False
cfg11sp["enable_checkpointing"] = False
cfg11sp["pl_params"]["precision"] = 16

In [7]:
cfg_list = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7]

In [10]:
cfg_list_p = [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p, cfg8p, cfg9p, cfg10p]

In [11]:
cfg_list_s = [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s]

In [12]:
cfg_list_sp = [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [13]:
cfg_list_all = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7] \
               [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p, cfg8p, cfg9p, cfg10p] + \
               [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s] + \
               [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]

In [54]:
#SVRなし最高
cfg_list = [cfg2, cfg3, cfg4, cfg5, cfg6, cfg7, cfg1p, cfg2p, cfg3p, cfg4p, cfg7p]

In [8]:
cfg_list = [cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7, cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg7p]

In [30]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp] # 実験開始

In [ ]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p] # <= subしたやつ 

In [62]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg2p, cfg3p, cfg4p, cfg7p]

In [ ]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg1s, cfg2s, cfg3s]

Model

In [8]:
def mcrmse(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:, i]
        y_pred = y_preds[:, i]
        score = metrics.mean_squared_error(y_true, y_pred, squared=False)  # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        )
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings


class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights=None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = (
            layer_weights
            if layer_weights is not None
            else nn.Parameter(
                torch.tensor(
                    [1] * (num_hidden_layers + 1 - layer_start), dtype=torch.float
                )
            )
        )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start :, :, :, :]
        weight_factor = (
            self.layer_weights.unsqueeze(-1)
            .unsqueeze(-1)
            .unsqueeze(-1)
            .expand(all_layer_embedding.size())
        )
        weighted_average = (weight_factor * all_layer_embedding).sum(
            dim=0
        ) / self.layer_weights.sum()
        return weighted_average


class AttentionHead(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionHead, self).__init__()
        self.att = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1),
            nn.Softmax(dim=1),
        )

    def forward(self, last_hidden_state):
        att_weights = self.att(last_hidden_state)
        feature = torch.sum(att_weights * last_hidden_state, dim=1)
        return feature


class TransformersModel(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.criterion = nn.__dict__[cfg["criterion"]]()

        # awp
        if cfg["awp"] is not None:
            self.automatic_optimization = False
            self.adv_param = cfg["awp"]["adv_param"]
            self.adv_lr = cfg["awp"]["adv_lr"]
            self.adv_eps = cfg["awp"]["adv_eps"]
            self.adv_step = cfg["awp"]["adv_step"]
            self.backup = {}
            self.backup_eps = {}
            self.awp_accumulate_grad_batches = cfg["awp"]["accumulate_grad_batches"]
            # self.awp_scaler = torch.cuda.amp.GradScaler(enabled=cfg["awp"]["amp"])
            if cfg["awp"]["gradient_clip_val"] is not None:
                self.awp_max_grad_norm = cfg["awp"]["gradient_clip_val"]
            else:
                self.awp_max_grad_norm = None
            self.awp_start_epoch = cfg["awp"]["start_epoch"]

        # model
        if cfg["mlm"]:
            print(f"../weights/mlm_model/{cfg['model_name']}")
            self.tr_config = AutoConfig.from_pretrained(
                f"../weights/mlm_model/{cfg['model_name']}", output_hidden_states=True
            )
        else:
            self.tr_config = AutoConfig.from_pretrained(
                cfg["model_name"], output_hidden_states=True
            )

        self.tr_config.hidden_dropout = 0.0
        self.tr_config.hidden_dropout_prob = 0.0
        self.tr_config.attention_dropout = 0.0
        self.tr_config.attention_probs_dropout_prob = 0.0

        if cfg["mlm"]:
            self.model = AutoModel.from_pretrained(
                f"../weights/mlm_model/{cfg['model_name']}", config=self.tr_config
            )
        elif cfg["pretrained"]:
            self.model = AutoModel.from_pretrained(
                cfg["model_name"], config=self.tr_config
            )
        else:
            self.model = AutoModel(self.tr_config)
        if self.cfg["transformers_params"]["gradient_checkpointing"]:
            self.model.gradient_checkpointing_enable()
        if cfg["preprocess"]:
            self.model.resize_token_embeddings(len(cfg["tokenizer"]))

        # header
        if cfg["header"] == "cls":
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "mean_pooling":
            self.pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "attention":
            self.attention = AttentionHead(self.tr_config.hidden_size)
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "weighted_average_pooling":
            layer_start = (
                self.tr_config.num_hidden_layers + 1
            ) - 4  # use last 4-layers
            self.wl_pool = WeightedLayerPooling(
                self.tr_config.num_hidden_layers,
                layer_start=layer_start,
                layer_weights=None,
            )
            self.m_pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "cls_concatenate":
            self.fc = nn.Linear(self.tr_config.hidden_size * 4, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "mean_pooling_concatenate":
            self.pool = MeanPooling()
            self.fc = nn.Linear(self.tr_config.hidden_size * 4, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "1dcnn":
            self.cnn1 = nn.Conv1d(
                self.tr_config.hidden_size, 256, kernel_size=2, padding=1
            )
            self.cnn2 = nn.Conv1d(256, 6, kernel_size=2, padding=1)
        elif cfg["header"] == "lstm":
            self.lstm = nn.LSTM(
                self.tr_config.hidden_size,
                self.tr_config.hidden_size,
                batch_first=True,
            )
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)
        elif cfg["header"] == "gru":
            self.gru = nn.GRU(
                self.tr_config.hidden_size,
                self.tr_config.hidden_size,
                batch_first=True,
            )
            self.fc = nn.Linear(self.tr_config.hidden_size, 6)
            self._init_weights(self.fc)

        # reinit same layers
        if cfg["transformers_params"]["reinit_layers"] is not None:
            print(f"reinit {cfg['transformers_params']['reinit_layers']} layers")
            self._reinit_layer(self.model)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.tr_config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def _reinit_layer(self, model):
        if "funnel" in self.cfg["model_name"]:
            for layer in model.decoder.layers[
                -self.cfg["transformers_params"]["reinit_layers"] :
            ]:
                for module in layer.modules():
                    if isinstance(module, nn.Linear):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.bias is not None:
                            module.bias.data.zero_()
                    elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                    elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)
        else:
            for layer in model.encoder.layer[
                -self.cfg["transformers_params"]["reinit_layers"] :
            ]:
                for module in layer.modules():
                    if isinstance(module, nn.Linear):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.bias is not None:
                            module.bias.data.zero_()
                    elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(
                            mean=0.0, std=model.config.initializer_range
                        )
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                    elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)

    def forward(self, inputs):
        outputs = self.model(**inputs)
        if self.cfg["header"] == "cls":
            last_hidden_states = outputs[0][:, 0]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(outputs, p=2, dim=1)
            else:
                outputs = self.fc(last_hidden_states)
        elif self.cfg["header"] == "mean_pooling":
            last_hidden_states = outputs[0]
            feature = self.pool(last_hidden_states, inputs["attention_mask"])
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "attention":
            last_hidden_states = outputs[0]
            feature = self.attention(last_hidden_states)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "weighted_average_pooling":
            all_hidden_states = torch.stack(outputs["hidden_states"])
            weighted_pooling_embeddings = self.wl_pool(all_hidden_states)
            feature = self.m_pool(weighted_pooling_embeddings, inputs["attention_mask"])
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "cls_concatenate":
            feature = torch.cat(
                [outputs["hidden_states"][-1 * i][:, 0] for i in range(1, 4 + 1)], dim=1
            )
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        if self.cfg["header"] == "mean_pooling_concatenate":
            features = []
            for i in range(1, 4 + 1):
                last_hidden_states = outputs["hidden_states"][-1 * i]
                feature = self.pool(last_hidden_states, inputs["attention_mask"])
                features.append(feature)
            feature = torch.cat(features, dim=1)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "1d_cnn":
            last_hidden_states = outputs["last_hidden_state"].permute(0, 2, 1)
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(last_hidden_states, p=2, dim=1)
            else:
                cnn_embeddings = F.relu(self.cnn1(last_hidden_states))
                cnn_embeddings = self.cnn2(cnn_embeddings)
                outputs, _ = torch.max(cnn_embeddings, 2)
        elif self.cfg["header"] == "lstm":
            feature, _ = self.lstm(outputs["last_hidden_state"], None)
            feature = feature[:, -1, :]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        elif self.cfg["header"] == "gru":
            feature, _ = self.gru(outputs["last_hidden_state"], None)
            feature = feature[:, -1, :]
            if self.cfg["retrieve_embeddings"]:
                outputs = F.normalize(feature, p=2, dim=1)
            else:
                outputs = self.fc(feature)
        return outputs

    def collate(self, inputs):
        # 一番長いtokenへ合わせる
        mask_len = int(inputs["attention_mask"].sum(axis=1).max())
        for k, v in inputs.items():
            inputs[k] = inputs[k][:, :mask_len]
        return inputs

    def training_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        if self.cfg["awp"] is not None:
            # awp step
            opt = self.optimizers()
            sch = self.lr_schedulers()

            # with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
            pred_y = self.forward(X)
            loss = self.criterion(pred_y, y)

            if self.awp_accumulate_grad_batches > 1:
                loss = loss / self.awp_accumulate_grad_batches
            # self.awp_scaler.scale(loss).backward()
            self.manual_backward(loss)

            if (batch_idx + 1) % self.awp_accumulate_grad_batches == 0:
                if self.trainer.current_epoch >= self.awp_start_epoch:
                    self._awp_save()
                    for _ in range(self.adv_step):
                        self._awp_attack_step()
                        # with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
                        pred_y = self.forward(X)
                        adv_loss = self.criterion(pred_y, y)
                        opt.zero_grad()
                        # self.awp_scaler.scale(adv_loss).backward()
                        self.manual_backward(adv_loss)
                    self._awp_restore()

                # self.awp_scaler.unscale_(opt)
                # torch.nn.utils.clip_grad_norm_(
                #    self.parameters(), self.awp_max_grad_norm
                # )
                # self.awp_scaler.step(opt)
                opt.step()
                # self.awp_scaler.update()
                opt.zero_grad()
                sch.step()
        else:
            # normal step
            pred_y = self.forward(X)
            loss = self.criterion(pred_y, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def training_epoch_end(self, outputs):
        loss_list = [x["loss"] for x in outputs]
        avg_loss = torch.stack(loss_list).mean()
        self.log("train_avg_loss", avg_loss, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        X, y = batch
        X = self.collate(X)
        # if self.cfg["awp"] is not None:
        #    with torch.cuda.amp.autocast(enabled=self.cfg["awp"]["amp"]):
        #        pred_y = self.forward(X)
        #        loss = self.criterion(pred_y, y)
        # else:
        pred_y = self.forward(X)
        loss = self.criterion(pred_y, y)
        return {"valid_loss": loss, "preds": pred_y, "targets": y}

    def validation_epoch_end(self, outputs):
        loss_list = [x["valid_loss"] for x in outputs]
        preds = torch.cat([x["preds"] for x in outputs], dim=0).cpu().detach().numpy()
        targets = (
            torch.cat([x["targets"] for x in outputs], dim=0).cpu().detach().numpy()
        )
        avg_loss = torch.stack(loss_list).mean()
        score, scores = mcrmse(targets, preds)
        self.log("valid_avg_loss", avg_loss, prog_bar=True)
        self.log("valid_score", score, prog_bar=True)
        self.log("valid_cohesion", scores[0], prog_bar=True)
        self.log("valid_syntax", scores[1], prog_bar=True)
        self.log("valid_vocabulary", scores[2], prog_bar=True)
        self.log("valid_phraseology	", scores[3], prog_bar=True)
        self.log("valid_grammar", scores[4], prog_bar=True)
        self.log("valid_conventions", scores[5], prog_bar=True)
        return avg_loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        X, _ = batch
        X = self.collate(X)
        pred_y = self.forward(X)
        return pred_y

    def get_scheduler(self, optimizer, num_train_steps):
        if self.cfg["transformers_params"]["scheduler"] == "linear":
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["warmup_ratio"]
                * num_train_steps,
                num_training_steps=num_train_steps,
            )
        elif self.cfg["transformers_params"]["scheduler"] == "cosine":
            scheduler = get_cosine_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.cfg["transformers_params"]["warmup_ratio"]
                * num_train_steps,
                num_training_steps=num_train_steps,
                num_cycles=self.cfg["transformers_params"]["num_cycles"],
            )
        return scheduler

    def configure_optimizers(self):
        # ↓ decayする層を選択 & header (decoder) には別の学習率を設定
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in self.named_parameters() if "model" not in n],
                "lr": self.cfg["transformers_params"]["decoder_lr"],
                "weight_decay": 0.0,
            },
        ]

        if "funnel" in self.cfg["model_name"]:
            layers = (
                [self.model.embeddings]
                + list(self.model.encoder.blocks)
                + list(self.model.decoder.layers)
            )
        else:
            layers = [self.model.embeddings] + list(self.model.encoder.layer)
        layers.reverse()
        lr = self.cfg["transformers_params"]["encoder_lr"]
        lr_decay = self.cfg["transformers_params"]["lr_decay_final"] ** (
            1.0 / len(layers)
        )
        for layer in layers:
            optimizer_parameters += [
                {
                    "params": [
                        p
                        for n, p in layer.named_parameters()
                        if not any(nd in n for nd in no_decay)
                    ],
                    "weight_decay": self.cfg["transformers_params"]["weight_decay"],
                    "lr": lr,
                },
                {
                    "params": [
                        p
                        for n, p in layer.named_parameters()
                        if any(nd in n for nd in no_decay)
                    ],
                    "weight_decay": 0.0,
                    "lr": lr,
                },
            ]
            lr *= lr_decay

        optimizer = optim.AdamW(
            optimizer_parameters, lr=self.cfg["transformers_params"]["encoder_lr"],
        )

        if self.cfg["awp"] is None:
            num_train_steps = self.trainer.estimated_stepping_batches
        else:
            num_train_steps = (
                self.trainer.estimated_stepping_batches
                / self.awp_accumulate_grad_batches
            )

        scheduler = self.get_scheduler(optimizer, num_train_steps)
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

    ###############################################################################
    # awp -------------------------------------------------------------------------
    ###############################################################################
    def on_before_optimizer_step(self, optimizer, optimizer_idx):
        if self.cfg["awp"] is not None:
            self.clip_gradients(
                optimizer,
                gradient_clip_val=self.awp_max_grad_norm,
                gradient_clip_algorithm=None,
            )

    def _awp_attack_step(self):
        e = 1e-6
        for name, param in self.named_parameters():
            if (
                param.requires_grad
                and param.grad is not None
                and self.adv_param in name
            ):
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]),
                        self.backup_eps[name][1],
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _awp_save(self):
        for name, param in self.named_parameters():
            if (
                param.requires_grad
                and param.grad is not None
                and self.adv_param in name
            ):
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _awp_restore(self):
        for name, param in self.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

In [9]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, cfg, X, y=None):
        self.cfg = cfg
        if y is None:
            self.X = X.values
            self.y = torch.zeros(len(self.X), dtype=torch.float32)
        else:
            self.X = X.values
            self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        X = self._prepare_input(self.X[index])
        y = self.y[index]
        return X, y

    def _prepare_input(self, X):
        if self.cfg["token_cut_head_and_tail"]:
            X = self.cut_head_and_tail(X)
        else:
            X = self.cfg["tokenizer"].encode_plus(
                X,
                return_tensors=None,
                add_special_tokens=True,
                max_length=self.cfg["tokenizer_params"]["max_length"],
                padding="max_length",
                truncation=True,
            )
        for k, v in X.items():
            X[k] = torch.tensor(v, dtype=torch.long)
        return X

    def cut_head_and_tail(self, text):
        # まずは限界を設定せずにトークナイズする
        max_len = self.cfg["tokenizer_params"]["max_length"]
        input_ids = self.cfg["tokenizer"].encode(text)
        n_token = len(input_ids)

        # トークン数が最大数と同じ場合
        if n_token == max_len:
            input_ids = input_ids
            attention_mask = [1 for _ in range(max_len)]
            token_type_ids = [1 for _ in range(max_len)]
        # トークン数が最大数より少ない場合
        elif n_token < max_len:
            pad = [1 for _ in range(max_len - n_token)]
            input_ids = input_ids + pad
            attention_mask = [1 if n_token > i else 0 for i in range(max_len)]
            token_type_ids = [1 if n_token > i else 0 for i in range(max_len)]
        # トークン数が最大数より多い場合
        else:
            harf_len = (max_len - 2) // 2
            _input_ids = input_ids[1:-1]
            input_ids = [0] + _input_ids[:harf_len] + _input_ids[-harf_len:] + [2]
            attention_mask = [1 for _ in range(max_len)]
            token_type_ids = [1 for _ in range(max_len)]

        d = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }

        return d

In [10]:
class TransformersRegressorInference:
    def __init__(self, cfg, weight_path=None):
        # tokenizer
        tokenizer = AutoTokenizer.from_pretrained(cfg["model_name"])
        if cfg["preprocess"]:
            tokenizer.add_tokens("[BR]", special_tokens=True)
        cfg["tokenizer"] = tokenizer

        self.model = TransformersModel(cfg)
        self.weight_path = weight_path
        self.cfg = cfg
        self.trainer = Trainer(**self.cfg["pl_params"])

    def predict(self, test_X):
        preds = []
        test_dataset = TableDataset(self.cfg, test_X)
        test_dataloader = torch.utils.data.DataLoader(
            test_dataset, **self.cfg["test_loader"],
        )
        preds = self.trainer.predict(
            self.model, dataloaders=test_dataloader, ckpt_path=self.weight_path
        )
        preds = torch.cat(preds, axis=0)
        preds = preds.cpu().detach().numpy()
        return preds

Data Preparation

In [11]:
# read csv
train = pd.read_csv("../data/input/train.csv")
test = pd.read_csv("../data/input/test.csv")
sub = pd.read_csv("../data/input/sample_submission.csv")

# split X/y
train_X = train["full_text"]
train_y = train.drop(["text_id", "full_text"], axis=1)
test_X = test["full_text"]

inference

In [12]:
def one_fold_valid(skf, cfg, valid_X, valid_y, fold_n):
    #print(f"[fold_{fold_n}]")
    #seed_everything(cfg["general"]["seed"], workers=True)

    if use_computed:
        valid_preds = joblib.load(f"../data/preds/valid_{cfg['general']['seed']}_{cfg['general']['save_name']}_{fold_n}.preds")
    else:
        model = TransformersRegressorInference(cfg, f"{cfg['save_weight_folder']}/last_epoch_fold{fold_n}.ckpt")
        valid_preds = model.predict(valid_X)
        
        del model
        gc.collect()
        torch.cuda.empty_cache()

    #score, scores = mcrmse(valid_y.values, valid_preds)
    #print(f"mcrmse_score:{score}, mcrmse_scores:{scores}")

    return valid_preds

In [13]:
# random seed setting
seed_everything(cfg1["general"]["seed"], workers=True)

Global seed set to 42


42

In [14]:
use_computed = True

optimize and blending

In [15]:
def choose(cfg_list):
    fold_list = range(cfg1["general"]["n_splits"])
    preds_list_cv = [[] for i in range(cfg1["general"]["n_splits"])]
    weights_list = [[] for _ in range(6)]
    skf = MultilabelStratifiedKFold(
        n_splits=cfg1["general"]["n_splits"], shuffle=True, random_state=cfg1["general"]["seed"]
    )
    for j, fold_n in enumerate(fold_list):
        preds_list = []
        _, valid_indices = list(skf.split(train_X, train_y))[fold_n]
        valid_X_cv, valid_y_cv = (
            train_X.iloc[valid_indices].reset_index(drop=True),
            train_y.iloc[valid_indices].reset_index(drop=True),
        )

        for i, cfg in enumerate(cfg_list):
            valid_preds = one_fold_valid(skf, cfg, valid_X_cv, valid_y_cv, fold_n)
            preds_list.append(valid_preds)
            #del valid_preds
            #gc.collect()

        #score, scores = mcrmse(valid_y_cv.values, np.mean(preds_list, axis=0))
        #print()
        #print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
        #for i, p in enumerate(preds_list):
            #score, scores = mcrmse(valid_y_cv.values, p)
            #print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")

        preds_list = np.array(preds_list)
        preds_list_cv[j] = preds_list
        cohesion = preds_list[:, :, 0]
        syntax = preds_list[:, :, 1]
        vocabulary = preds_list[:, :, 2]
        phraseology = preds_list[:, :, 3]
        grammar = preds_list[:, :, 4]
        conventions = preds_list[:, :, 5]
        target_list = [cohesion, syntax, vocabulary, phraseology, grammar, conventions]

        for t_idx, target in enumerate(target_list):

            def f(x):
                pred = np.zeros_like(target[0])
                for i, p in enumerate(target):
                    pred += p * x[i]
                score = metrics.mean_squared_error(
                    valid_y_cv.values[:, t_idx], pred, squared=False
                )
                return score

            init_state = np.ones((len(target))) / len(target)
            bounds = [(0.0, 1.0)] * len(target)
            result = minimize(f, init_state, method="Nelder-Mead", bounds=bounds)
            #print(f"optimized_corr:{result['fun']}")

            weights = [[0] for _ in range(len(target))]
            for i in range(len(target)):
                weights[i] = result["x"][i]
            weights_list[t_idx].append(weights)
            #print(f"weights:{weights}")

    avg_weights_cohesion = np.mean(weights_list[0], axis=0)
    avg_weights_syntax = np.mean(weights_list[1], axis=0)
    avg_weights_vocabulary = np.mean(weights_list[2], axis=0)
    avg_weights_phraseology = np.mean(weights_list[3], axis=0)
    avg_weights_grammar = np.mean(weights_list[4], axis=0)
    avg_weights_conventions = np.mean(weights_list[5], axis=0)

    #print()
    #print(f"cohesion averaged_weights:{avg_weights_cohesion}")
    #print(f"syntax averaged_weights:{avg_weights_syntax}")
    #print(f"vocabulary averaged_weights:{avg_weights_vocabulary}")
    #print(f"phraseology averaged_weights:{avg_weights_phraseology}")
    #print(f"grammar averaged_weights:{avg_weights_grammar}")
    #print(f"conventions averaged_weights:{avg_weights_conventions}")

    score_cv = []
    #scores_cv = []
    preds_list_valid_cv = [[] for i in range(cfg1["general"]["n_splits"])]
    for j, preds in enumerate(preds_list_cv):

        _, valid_indices = list(skf.split(train_X, train_y))[j]
        valid_X_cv, valid_y_cv = (
            train_X.iloc[valid_indices].reset_index(drop=True),
            train_y.iloc[valid_indices].reset_index(drop=True),
        )
        
        # blending
        preds_list_valid_cv[j] = np.zeros(preds[0].shape)
        for i in range(preds.shape[0]):
            preds_list_valid_cv[j][:, 0] += preds[i, :, 0] * avg_weights_cohesion[i]
            preds_list_valid_cv[j][:, 1] += preds[i, :, 1] * avg_weights_syntax[i]
            preds_list_valid_cv[j][:, 2] += preds[i, :, 2] * avg_weights_vocabulary[i]
            preds_list_valid_cv[j][:, 3] += preds[i, :, 3] * avg_weights_phraseology[i]
            preds_list_valid_cv[j][:, 4] += preds[i, :, 4] * avg_weights_grammar[i]
            preds_list_valid_cv[j][:, 5] += preds[i, :, 5] * avg_weights_conventions[i]
            
        #score, scores = mcrmse(valid_y_cv.values, np.mean(preds, axis=0))
        #print()
        #print(f"simple_mean: mcrmse_score:{score}, mcrmse_scores:{scores}")
        #for i, p in enumerate(preds):
            #score, scores = mcrmse(valid_y_cv.values, p)
            #print(f"cfg_{i}: mcrmse_score:{score}, mcrmse_scores:{scores}")
            
        score, scores = mcrmse(valid_y_cv.values, preds_list_valid_cv[j])
        #print(f"optimized: mcrmse_score:{score}, mcrmse_scores:{scores}")
        score_cv.append(score)
        #scores_cv.append(scores)

    #print()
    #print(f"optimized mean_cv score:{np.mean(score_cv)}")
    #print(f"optimized mean_cv score:{np.mean(scores_cv, axis=0)}")
    return np.mean(score_cv)

In [57]:
cfg_list = ([cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8p, cfg9p] \
+ [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]).copy()

In [58]:
cfg_list_retrieve = cfg_list.copy()

In [59]:
current_score = choose(cfg_list)
print(f"current_score: {current_score}")
repeat_score = [[] for _ in range(len(cfg_list))]
for i in range(len(cfg_list_retrieve)):
    cfg_list_repeat = cfg_list_retrieve.copy()
    del cfg_list_repeat[i]
    repeat_score[i] = choose(cfg_list_repeat)
    print("retrieve: ", cfg_list_retrieve[i]["general"]["save_name"])
    print("score: ", repeat_score[i])
current_score_ = np.min(repeat_score)
retrieve_idx = np.argmin(repeat_score)
retrieve_model = cfg_list_retrieve[retrieve_idx]["general"]["save_name"]
print()
print(f"score: {current_score} -> {current_score_}")
print(f"retrieve_model: {retrieve_model}, index: {retrieve_idx}")

current_score: 0.44411874147652836
retrieve:  deberta_v3_base_512_reinit
score:  0.4440368526144952


KeyboardInterrupt: 

In [ ]:
# score: 0.44365529480865284 -> 0.4435950812754264
# retrieve_model: deberta_v3_base_512_reinit, index: 0
# -> cfg_list = [cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg6s]

# score: 0.4435950812754264 -> 0.44358263929363534
# retrieve_model: deberta_v3_large_512_reinit_pseudo3, index: 5
# -> cfg_list = [cfg2, cfg5, cfg6, cfg7, cfg1p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg6s]

# score: 0.44358263929363534 -> 0.4435757923511847
# retrieve_model: deberta_v3_large_512_reinit_svr_pseudo3, index: 9
# -> cfg_list = [cfg2, cfg5, cfg6, cfg7, cfg1p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg5sp, cfg8s, cfg3s, cfg6s]


In [63]:
cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg6s]

In [64]:
added_cfg_list = [cfg2s, cfg4s, cfg7s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s]
# 重いので除外 cfg5s, cfg1s

In [65]:
from tqdm import tqdm

current_score = choose(cfg_list)
print(f"current_score: {current_score}")
repeat_score = [[] for _ in range(len(added_cfg_list))]
for i in tqdm(range(len(added_cfg_list))):
    cfg_list_repeat = cfg_list + [added_cfg_list[i]]
    repeat_score[i] = choose(cfg_list_repeat)
    print(i)
    print("add: ", added_cfg_list[i]["general"]["save_name"])
    print("score: ", repeat_score[i])
current_score_ = np.min(repeat_score)
add_idx = np.argmin(repeat_score)
add_model = added_cfg_list[add_idx]["general"]["save_name"]
print()
print(f"score: {current_score} -> {current_score_}")
print(f"add_model: {add_model}, index: {add_idx}")


current_score: 0.44365529480865284


 10%|█         | 1/10 [00:38<05:43, 38.22s/it]


0
add:  bigbird_roberta_large_4096_svr
score:  0.4437192172441923


 20%|██        | 2/10 [01:16<05:03, 37.97s/it]


1
add:  deberta_v2_xlarge_512_svr
score:  0.44368133216181427


...


7
add:  longformer_large_4096_svr
score:  0.4437240974876023


 90%|█████████ | 9/10 [05:40<00:37, 37.83s/it]


8
add:  muppet_large_512_svr
score:  0.4437030795536943


100%|██████████| 10/10 [06:18<00:00, 37.88s/it]


9
add:  roberta_large_512_svr
score:  0.4437242722782311

score: 0.44365529480865284 -> 0.44365578495344976
add_model: deberta_v3_base_512_svr, index: 2


In [ ]:
#score: 0.4439758496123359 -> 0.4437593246604853
#add_model: deberta_v3_large_4096_svr, index: 7

#score: 0.4437593246604853 -> 0.44368171586630173
#add_model: deberta_large_512_svr, index: 2
# -> cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s]

#score: 0.44368171586630173 -> 0.44365529480865284
#add_model: deberta_v3_base_4096_svr
# -> cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg6s]






#score: 0.44368171586630173 -> 0.44364656641620137
#add_model: deberta_v2_xxlarge_512_svr, index: 3 <- あんま追加したくない、、
# -> cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg5s]

#score: 0.44364656641620137 -> 0.4436286575596469
#add_model: deberta_v3_base_4096_svr, index: 3
# -> cfg_list = [cfg1, cfg2, cfg5, cfg6, cfg7, cfg1p, cfg3p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg4sp, cfg5sp, cfg8s, cfg3s, cfg5s, cfg6s]



scoreが下がらなくなるまでループ

In [23]:
cfg_list_retrieve = ([cfg2, cfg5, cfg6, cfg7, cfg1p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg5sp, cfg8s, cfg3s, cfg6s] + [cfg6sp, cfg7sp, cfg8sp, cfg9sp, cfg10sp, cfg11sp]).copy()

In [61]:
"""
cfg_list_retrieve = ([cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7] + \
                 [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg8p, cfg9p, cfg10p] + \
                 [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s] + \
                 [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]).copy()
"""

'\ncfg_list_retrieve = ([cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7] +                  [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg8p, cfg9p, cfg10p] +                  [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s] +                  [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]).copy()\n'

In [24]:
current_score = choose(cfg_list_retrieve)
print(f"current_score: {current_score}")

best_score = np.inf
while current_score < best_score:
    best_score = current_score
    best_cfg = cfg_list_retrieve.copy()
    repeat_score = [[] for _ in range(len(cfg_list_retrieve))]
    for i in tqdm(range(len(cfg_list_retrieve))):
        cfg_list_repeat = cfg_list_retrieve.copy()
        del cfg_list_repeat[i]
        repeat_score[i] = choose(cfg_list_repeat)
        #print("retrieve: ", cfg_list_retrieve[i]["general"]["save_name"])
        #print("score: ", repeat_score[i])
    current_score = np.min(repeat_score)
    retrieve_idx = np.argmin(repeat_score)
    retrieve_model = cfg_list_retrieve[retrieve_idx]["general"]["save_name"]
    print()
    print(f"score: {best_score} -> {current_score}")
    print(f"retrieve_model: {retrieve_model}, index: {retrieve_idx}")
    del cfg_list_retrieve[retrieve_idx]

current_score: 0.44376791841734936


100%|██████████| 19/19 [06:37<00:00, 20.91s/it]



score: 0.44376791841734936 -> 0.44366329666050924
retrieve_model: deberta_v3_base_4096_reinit_svr_pseudo_mean_pooling_concatenate, index: 18


100%|██████████| 18/18 [05:51<00:00, 19.52s/it]



score: 0.44366329666050924 -> 0.44363836989695715
retrieve_model: deberta_v3_base_512_reinit_svr_pseudo_mean_pooling_concatenate, index: 15


...


score: 0.44363836989695715 -> 0.44361097844447855
retrieve_model: deberta_v3_base_4096_reinit_svr_pseudo_wap, index: 15


100%|██████████| 16/16 [04:26<00:00, 16.64s/it]



score: 0.44361097844447855 -> 0.4435978348259003
retrieve_model: deberta_v3_base_4096_reinit_svr_pseudo_wap_all, index: 15


100%|██████████| 15/15 [03:48<00:00, 15.23s/it]


score: 0.4435978348259003 -> 0.44361031531477335
retrieve_model: deberta_v3_base_512_reinit_pseudo3, index: 4


In [52]:
for i in best_cfg:
    print(i["general"]["save_name"])

deberta_v3_base_4096_reinit
deberta_v3_large_512_reinit
deberta_v3_large_4096_reinit
deberta_large_512_reinit
muppet_large_512_reinit
deberta_xlarge_512_reinit
deberta_v3_base_512_reinit_pseudo3
deberta_v3_base_4096_reinit_pseudo
deberta_v3_large_512_reinit_pseudo3
deberta_v3_large_4096_reinit_pseudo
deberta_xlarge_512_reinit_pseudo


In [17]:
cfg_list_base = ([cfg2, cfg5, cfg6, cfg7, cfg1p, cfg4p, cfg7p, cfg2sp, cfg3sp, cfg5sp, cfg8s, cfg3s, cfg6s] + [cfg6sp, cfg7sp, cfg8sp, cfg9sp, cfg10sp, cfg11sp]).copy()

In [20]:
added_cfg_list = [cfg6sp, cfg7sp, cfg8sp, cfg9sp, cfg10sp, cfg11sp]

In [22]:
"""
added_cfg_list = ([cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7] + \
                 [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg8p, cfg9p, cfg10p] + \
                 [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s] + \
                 [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]).copy()
"""

'\nadded_cfg_list = ([cfg1, cfg2, cfg3, cfg4, cfg5, cfg6, cfg7] +                  [cfg1p, cfg2p, cfg3p, cfg4p, cfg5p, cfg6p, cfg8p, cfg9p, cfg10p] +                  [cfg1s, cfg2s, cfg3s, cfg4s, cfg5s, cfg6s, cfg7s, cfg8s, cfg9s, cfg10s, cfg11s, cfg12s, cfg13s, cfg14s, cfg15s] +                  [cfg1sp, cfg2sp, cfg3sp, cfg4sp, cfg5sp]).copy()\n'

In [22]:
current_score = choose(cfg_list_base)
print(f"current_score: {current_score}")

best_score = np.inf
while current_score < best_score:
    best_score = current_score
    best_cfg = cfg_list_base.copy()
    repeat_score = [[] for _ in range(len(added_cfg_list))]
    for i in tqdm(range(len(added_cfg_list))):
        cfg_list_repeat = cfg_list_base + [added_cfg_list[i]]
        repeat_score[i] = choose(cfg_list_repeat)
        #print(i)
        #print("add: ", added_cfg_list[i]["general"]["save_name"])
        #print("score: ", repeat_score[i])
    current_score = np.min(repeat_score)
    add_idx = np.argmin(repeat_score)
    add_model = added_cfg_list[add_idx]["general"]["save_name"]
    print()
    print(f"score: {best_score} -> {current_score}")
    print(f"add_model: {add_model}, index: {add_idx}")
    cfg_list_base.append(added_cfg_list[add_idx])
    del added_cfg_list[add_idx]

current_score: 0.4435757923511847


100%|██████████| 6/6 [01:31<00:00, 15.29s/it]


score: 0.4435757923511847 -> 0.4436024923441589
add_model: deberta_v3_base_4096_reinit_svr_pseudo_wap, index: 3


In [25]:
for i in best_cfg:
    print(i["general"]["save_name"])

deberta_v3_base_512_reinit
deberta_v3_base_4096_reinit
deberta_large_512_reinit
muppet_large_512_reinit
deberta_xlarge_512_reinit
deberta_v3_base_512_reinit_pseudo3
deberta_v3_large_512_reinit_pseudo3
deberta_v3_large_4096_reinit_pseudo
deberta_xlarge_512_reinit_pseudo
deberta_v3_base_512_reinit_svr_pseudo
deberta_v3_large_4096_reinit_svr_pseudo
deberta_v3_large_512_reinit_svr_pseudo3
deberta_xlarge_512_reinit_svr_pseudo
deberta_v3_large_4096_svr
deberta_large_512_svr
deberta_v2_xxlarge_512_svr
deberta_v3_base_4096_svr
